# Imports

In [10]:
import pandas as pd
import numpy as np
from scipy import stats
from itertools import product
from scipy.stats import laplace
    
df = pd.read_csv("movie_metadata.csv")
for i in df.index:
    if(df.isna().at[i,"gross"] or df.isna().at[i,"language"] or df.isna().at[i,"country"]): df = df.drop(i)

In [11]:
def topgross(df,eplison):
    if(df.shape[0] == 1): sensibilidade = 0 
    else : sensibilidade = np.max(df["gross"].values) - np.max(np.delete(df["gross"].values,np.where(df["gross"].values == [np.max(df["gross"].values)])))
    x = abs(laplace.rvs(loc = 0, scale = sensibilidade/eplison))
    return np.max(df["gross"].values) + x

# Score for the first Query

In [12]:
def score_gross(df,eplison):
    Dict = {}
    
    max_value = topgross(df,eplison)    
    #A = [df["movie_title"].values,df["movie_title"].values]
    #A = list(product(*A))
    
    #for mov1, mov2 in A:
    #    if(mov1 == mov2):
    #        value = df[df["movie_title"] == mov1]["gross"].values[0]
    #        Dict[mov1,mov2] = -abs(max_value - value)/max_value
    #    else: Dict[mov1,mov2] = 0
    
    for movie_title in df["movie_title"].values:
        value = df[df["movie_title"] == movie_title]["gross"].values[0]
        Dict[movie_title] = -abs(max_value - value)/max_value
        
    return Dict

In [13]:
def consulta1(df, eplison):
     
    fscore_df = score_gross(df,0.01*eplison) 
    sensibilidade = min(fscore_df.values())*(-1)
    
    total_sum = 0
    for movie_title in df["movie_title"].values:
        if(sensibilidade!=0): total_sum += np.e**(0.99*eplison*fscore_df[movie_title]/(2*sensibilidade))

    xk = df.index
    pk = []
    for idnx in df.index:
        if(sensibilidade != 0): pk.append(np.e**(0.99*eplison*fscore_df[df.at[idnx,"movie_title"]]/(2*sensibilidade))/total_sum)
        else: pk.append(1)
            
            
    custm = stats.rv_discrete(name='custm', values=(xk, pk))
    
    
    return df.at[custm.rvs(size=1)[0], "movie_title"],sensibilidade

In [14]:
def consulta2(df, eplison):
    result = []
        
    language_set = set(df["language"].values)
    for language in language_set:
        df_language = df[df["language"] == language]
        result.append(consulta1(df_language,eplison))
    
    return result

In [15]:
def topcount_country(df, eplison):
    group_by = df.groupby(["country"], as_index=False).count()
    if(df.shape[0] == 1): sensibilidade = 0
    else: sensibilidade = np.max(df["color"].values) - np.max(np.delete(df["color"].values,np.where(df["color"].values == [np.max(df["color"].values)])))
    x = abs(laplace.rvs(loc = 0, scale = sensibilidade/eplison))
    return np.max(df["color"].values) + x

# Score For the Third Query

In [16]:
def score_count(df, eplison):
    Dict = {}
    group_by = df.groupby(["country"], as_index=False).count()
    max_value = topcount_country(group_by, 0.01*eplison)
    
    for movie_country in df["country"].values:
        Dict[movie_country] = -abs(max_value-group_by[group_by["country"]== movie_country]["color"].values[0])/max_value
        
    return Dict

In [17]:
def consulta3_aux(df, eplison):
    
    fscore_df = score_count(df,eplison)  
    sensibilidade = min(fscore_df.values())*(-1)
    
    
    total_sum = 0
    for movie_country in df["country"].values:
        if(sensibilidade!=0): total_sum += np.e**(0.99*eplison*fscore_df[movie_country]/(2*sensibilidade))
    
   
    xk = df.index
    pk = []
    for idnx in df.index:
        if(sensibilidade != 0): pk.append(np.e**(0.99*eplison*fscore_df[df.at[idnx,"country"]]/(2*sensibilidade))/total_sum)
        else: pk.append(1)
            
    custm = stats.rv_discrete(name='custm', values=(xk, pk))
    
    
    return df.at[custm.rvs(size=1)[0], "country"],sensibilidade
    

In [29]:
def consulta3(df, eplison):
    result = []
    sens = []
    r1 = consulta3_aux(df, eplison/3)
    result.append(r1[0])
    sens.append(r1[1])
    r2 = consulta3_aux(df[df["country"]!=r1[0]], eplison/3)
    result.append(r2[0])
    sens.append(r2[1])
    r3 = consulta3_aux(df[df["country"]!=r1[0]][df[df["country"]!=r1[0]]["country"]!=r2[0]], eplison/3)
    result.append(r3[0])
    sens.append(r3[1])
    return result,sens

# Parameters

In [71]:
eplisons = [0.1/3,1/3,10/3]

# Consulta 1

In [72]:
c1_results = []
c1_sens = []

for eplison in eplisons:
    c1 = consulta1(df,eplison)
    c1_results.append(c1[0])
    c1_sens.append(c1[1])

In [73]:
c1_results[0]

'Allegiant\xa0'

# Consulta 2

In [74]:
c2_results = []
c2_sens = []

for eplison in eplisons:
    c2 = consulta2(df,eplison)
    c2 =np.array(c2)
    c2_results.append(c2[:,0])
    c2_sens.append(c2[:,1])

# Consulta 3

In [75]:
c3_results = []
c3_sens = []

for eplison in eplisons:
    c3 = consulta3(df,eplison)
    c3 =np.array(c3)
    c3_results.append(c3[0])
    c3_sens.append(c3[1])

In [76]:
c3

array([['USA', 'UK', 'Japan'],
       ['0.9999973611916023', '0.9999398075854483', '0.9996416334001909']],
      dtype='<U18')

In [77]:
data = []
budgets = [0.1,1,10]
i = 0
for budget in budgets:
    data.append([budget,c1_results[i],c2_results[i],c3_results[i],c1_sens[i],c2_sens[i],c3_sens[i]])
    i += 1

In [78]:
results = pd.DataFrame(data, columns = [ 'budget', 'result q1', 'result q2', 'result q3', 
                                        'sens q1', 'sens q2', 'sens q3'])

In [79]:
results

,budget,result q1,result q2,result q3,sens q1,sens q2,sens q3
0,0.1,Allegiant,"[Hero , Soul Kitchen , 4 Months, 3 Weeks and 2...","[USA, UK, France]",1.0,"[0.9999999796642788, 0.9999955209406031, 0.0, ...","[0.9999999339612632, 0.9999997301399438, 0.999..."
1,1.0,The Game,"[A Woman, a Gun and a Noodle Shop , The Baader...","[USA, UK, Germany]",1.0,"[0.9999997738873245, 0.9998171586044348, 0.0, ...","[0.9999997137925601, 0.999991585869927, 0.9999..."
2,10.0,G.I. Jane,"[Hero , Run Lola Run , 4 Months, 3 Weeks and 2...","[USA, UK, Japan]",1.0,"[0.9999973883582122, 0.9995188988210817, 0.0, ...","[0.9999973611916023, 0.9999398075854483, 0.999..."


In [80]:
results.to_csv('result.csv',index=False,sep=',')

In [9]:
df.sort_values(by = "gross", ascending=False)["movie_title"].to_string()

"0                                                 Avatar\xa0\n26                                               Titanic\xa0\n29                                        Jurassic World\xa0\n17                                          The Avengers\xa0\n794                                         The Avengers\xa0\n66                                       The Dark Knight\xa0\n240            Star Wars: Episode I - The Phantom Menace\xa0\n3024                  Star Wars: Episode IV - A New Hope\xa0\n8                                Avengers: Age of Ultron\xa0\n3                                  The Dark Knight Rises\xa0\n582                                              Shrek 2\xa0\n3080                          E.T. the Extra-Terrestrial\xa0\n186                      The Hunger Games: Catching Fire\xa0\n13            Pirates of the Caribbean: Dead Man's Chest\xa0\n509                                        The Lion King\xa0\n43                                           Toy Story 3\xa0\n32     